https://gist.github.com/DollarAkshay/e6fe84fdf721db2731948e8461092e5a

In [14]:
import time, math, random, bisect, copy
import gym
import numpy as np
import csv

In [17]:
GAME = 'LunarLander-v2'
MAX_STEPS = 200
MAX_GENERATIONS = 2000
POPULATION_COUNT = 100
MUTATION_RATE = 0.01

In [18]:
class NeuralNet : 
    def __init__(self, nodeCount):     
        self.fitness = 0.0
        self.nodeCount = nodeCount
        self.weights = []
        self.biases = []
        for i in range(len(nodeCount) - 1):
            self.weights.append( np.random.uniform(low=-1, high=1, size=(nodeCount[i], nodeCount[i+1])).tolist() )
            self.biases.append( np.random.uniform(low=-1, high=1, size=(nodeCount[i+1])).tolist())


    def printWeightsandBiases(self):
        
        print("--------------------------------")
        print("Weights :\n[", end="")
        for i in range(len(self.weights)):
            print("\n [ ", end="")
            for j in range(len(self.weights[i])):
                if j!=0:
                    print("\n   ", end="")
                print("[", end="")
                for k in range(len(self.weights[i][j])):
                    print(" %5.2f," % (self.weights[i][j][k]), end="")
                print("\b],", end="")
            print("\b ],")
        print("\n]")

        print("\nBiases :\n[", end="")
        for i in range(len(self.biases)):
            print("\n [ ", end="")
            for j in range(len(self.biases[i])):
                    print(" %5.2f," % (self.biases[i][j]), end="")
            print("\b],", end="")
        print("\b \n]\n--------------------------------\n")
  
    def getOutput(self, input):
        output = input
        for i in range(len(self.nodeCount)-1):
            output = np.reshape( np.matmul(output, self.weights[i]) + self.biases[i], (self.nodeCount[i+1]))
        return np.argmax(output)


class Population :
    def __init__(self, populationCount, mutationRate, nodeCount):
        self.nodeCount = nodeCount
        self.popCount = populationCount
        self.m_rate = mutationRate
        self.population = [ NeuralNet(nodeCount) for i in range(populationCount)]


    def createChild(self, nn1, nn2):
        
        child = NeuralNet(self.nodeCount)
        for i in range(len(child.weights)):
            for j in range(len(child.weights[i])):
                for k in range(len(child.weights[i][j])):
                    if random.random() < self.m_rate:
                        child.weights[i][j][k] = random.uniform(-1,1)
                    else: 
                        child.weights[i][j][k] = (nn1.weights[i][j][k] + nn2.weights[i][j][k])/2
                        
                        

        for i in range(len(child.biases)):
            for j in range(len(child.biases[i])):
                if random.random() < self.m_rate:
                    child.biases[i][j] = random.uniform(-1,1)
                else: 
                    child.biases[i][j] = (nn1.biases[i][j] + nn2.biases[i][j])/2
                        
        return child


    def createNewGeneration(self, bestNN):    

        nextGen = []
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        for i in range(self.popCount):
            if random.random() < float(self.popCount-i)/self.popCount:
                nextGen.append(copy.deepcopy(self.population[i]));

        fitnessSum = [0]
        minFit = min([i.fitness for i in nextGen])
        for i in range(len(nextGen)):
            fitnessSum.append(fitnessSum[i]+(nextGen[i].fitness-minFit)**4)
        

        while(len(nextGen) < self.popCount):
            r1 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            r2 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            nn1 = nextGen[bisect.bisect_right(fitnessSum, r1)-1]
            nn2 = nextGen[bisect.bisect_right(fitnessSum, r2)-1]
            nextGen.append( self.createChild(nn1, nn2) )
        self.population.clear()
        self.population = nextGen


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))
            
env = gym.make(GAME)
observation = env.reset()
in_dimen = env.observation_space.shape[0]
out_dimen = env.action_space.n
obsMin = env.observation_space.low
obsMax = env.observation_space.high
actionMin = 0
actionMax = env.action_space.n
pop = Population(POPULATION_COUNT, MUTATION_RATE, [in_dimen, 13, 8, 13, out_dimen])
bestNeuralNets = []

with open("GA_LUNAR_train_scores.csv", "w") as csvfile:
            header = ["episode", "score"]
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(header)
            
            for gen in range(MAX_GENERATIONS):
                genAvgFit = 0.0
                maxFit = -100000000
                maxNeuralNet = None
                for nn in pop.population:
                    totalReward = 0
                    for step in range(MAX_STEPS):
                        action = nn.getOutput(observation)
                        observation, reward, done, info = env.step(action)
                        totalReward += reward
                        if done:
                            observation = env.reset()
                            break

                    nn.fitness = totalReward
                    genAvgFit += nn.fitness
                    if nn.fitness > maxFit :
                        maxFit = nn.fitness
                        maxNeuralNet = copy.deepcopy(nn);

                bestNeuralNets.append(maxNeuralNet)
                genAvgFit/=pop.popCount
                print("Generation : %3d |  Avg Fitness : %5.0f  |  Max Fitness : %5.0f  " % (gen+1, genAvgFit, maxFit) )
                writer.writerow([gen+1, genAvgFit])
                pop.createNewGeneration(maxNeuralNet)

Generation :   1 |  Avg Fitness :  -447  |  Max Fitness :    37  
Generation :   2 |  Avg Fitness :  -367  |  Max Fitness :    -1  
Generation :   3 |  Avg Fitness :  -329  |  Max Fitness :    40  
Generation :   4 |  Avg Fitness :  -294  |  Max Fitness :    -5  
Generation :   5 |  Avg Fitness :  -261  |  Max Fitness :    35  
Generation :   6 |  Avg Fitness :  -269  |  Max Fitness :    33  
Generation :   7 |  Avg Fitness :  -229  |  Max Fitness :   122  
Generation :   8 |  Avg Fitness :  -204  |  Max Fitness :    29  
Generation :   9 |  Avg Fitness :  -162  |  Max Fitness :     5  
Generation :  10 |  Avg Fitness :  -146  |  Max Fitness :     9  
Generation :  11 |  Avg Fitness :  -135  |  Max Fitness :    33  
Generation :  12 |  Avg Fitness :  -128  |  Max Fitness :    24  
Generation :  13 |  Avg Fitness :  -130  |  Max Fitness :    59  
Generation :  14 |  Avg Fitness :  -138  |  Max Fitness :    39  
Generation :  15 |  Avg Fitness :  -129  |  Max Fitness :    21  
Generation

Generation : 127 |  Avg Fitness :  -114  |  Max Fitness :   256  
Generation : 128 |  Avg Fitness :  -156  |  Max Fitness :   263  
Generation : 129 |  Avg Fitness :  -171  |  Max Fitness :   245  
Generation : 130 |  Avg Fitness :  -156  |  Max Fitness :    54  
Generation : 131 |  Avg Fitness :  -170  |  Max Fitness :   284  
Generation : 132 |  Avg Fitness :  -169  |  Max Fitness :    53  
Generation : 133 |  Avg Fitness :  -147  |  Max Fitness :   241  
Generation : 134 |  Avg Fitness :  -143  |  Max Fitness :   107  
Generation : 135 |  Avg Fitness :  -138  |  Max Fitness :   227  
Generation : 136 |  Avg Fitness :  -156  |  Max Fitness :   168  
Generation : 137 |  Avg Fitness :  -185  |  Max Fitness :    52  
Generation : 138 |  Avg Fitness :  -156  |  Max Fitness :   120  
Generation : 139 |  Avg Fitness :  -140  |  Max Fitness :   234  
Generation : 140 |  Avg Fitness :  -125  |  Max Fitness :   272  
Generation : 141 |  Avg Fitness :  -142  |  Max Fitness :   129  
Generation

Generation : 252 |  Avg Fitness :  -121  |  Max Fitness :   252  
Generation : 253 |  Avg Fitness :  -110  |  Max Fitness :   221  
Generation : 254 |  Avg Fitness :  -105  |  Max Fitness :   273  
Generation : 255 |  Avg Fitness :  -113  |  Max Fitness :   183  
Generation : 256 |  Avg Fitness :  -124  |  Max Fitness :   157  
Generation : 257 |  Avg Fitness :   -96  |  Max Fitness :   255  
Generation : 258 |  Avg Fitness :  -162  |  Max Fitness :   238  
Generation : 259 |  Avg Fitness :  -101  |  Max Fitness :   278  
Generation : 260 |  Avg Fitness :  -133  |  Max Fitness :   135  
Generation : 261 |  Avg Fitness :  -107  |  Max Fitness :   258  
Generation : 262 |  Avg Fitness :  -121  |  Max Fitness :   245  
Generation : 263 |  Avg Fitness :   -88  |  Max Fitness :   273  
Generation : 264 |  Avg Fitness :  -121  |  Max Fitness :   173  
Generation : 265 |  Avg Fitness :  -125  |  Max Fitness :   231  
Generation : 266 |  Avg Fitness :  -127  |  Max Fitness :    36  
Generation

Generation : 377 |  Avg Fitness :   -94  |  Max Fitness :   295  
Generation : 378 |  Avg Fitness :   -65  |  Max Fitness :   291  
Generation : 379 |  Avg Fitness :   -95  |  Max Fitness :   267  
Generation : 380 |  Avg Fitness :   -74  |  Max Fitness :   263  
Generation : 381 |  Avg Fitness :   -89  |  Max Fitness :   267  
Generation : 382 |  Avg Fitness :   -74  |  Max Fitness :   262  
Generation : 383 |  Avg Fitness :   -77  |  Max Fitness :   268  
Generation : 384 |  Avg Fitness :   -45  |  Max Fitness :   253  
Generation : 385 |  Avg Fitness :   -66  |  Max Fitness :   283  
Generation : 386 |  Avg Fitness :  -101  |  Max Fitness :   274  
Generation : 387 |  Avg Fitness :   -79  |  Max Fitness :   274  
Generation : 388 |  Avg Fitness :   -64  |  Max Fitness :   297  
Generation : 389 |  Avg Fitness :   -94  |  Max Fitness :   290  
Generation : 390 |  Avg Fitness :   -87  |  Max Fitness :   259  
Generation : 391 |  Avg Fitness :   -76  |  Max Fitness :   254  
Generation

Generation : 503 |  Avg Fitness :  -123  |  Max Fitness :   130  
Generation : 504 |  Avg Fitness :  -101  |  Max Fitness :   148  
Generation : 505 |  Avg Fitness :  -117  |  Max Fitness :   255  
Generation : 506 |  Avg Fitness :  -106  |  Max Fitness :   113  
Generation : 507 |  Avg Fitness :   -94  |  Max Fitness :   182  
Generation : 508 |  Avg Fitness :   -90  |  Max Fitness :   159  
Generation : 509 |  Avg Fitness :   -85  |  Max Fitness :   179  
Generation : 510 |  Avg Fitness :   -93  |  Max Fitness :   169  
Generation : 511 |  Avg Fitness :  -112  |  Max Fitness :   154  
Generation : 512 |  Avg Fitness :  -101  |  Max Fitness :   186  
Generation : 513 |  Avg Fitness :   -90  |  Max Fitness :   274  
Generation : 514 |  Avg Fitness :  -123  |  Max Fitness :   124  
Generation : 515 |  Avg Fitness :  -133  |  Max Fitness :   150  
Generation : 516 |  Avg Fitness :  -102  |  Max Fitness :   212  
Generation : 517 |  Avg Fitness :   -80  |  Max Fitness :   183  
Generation

...

Generation : 1495 |  Avg Fitness :  -142  |  Max Fitness :   213  
Generation : 1496 |  Avg Fitness :  -109  |  Max Fitness :   188  
Generation : 1497 |  Avg Fitness :  -142  |  Max Fitness :   135  
Generation : 1498 |  Avg Fitness :  -111  |  Max Fitness :   170  
Generation : 1499 |  Avg Fitness :  -111  |  Max Fitness :   160  
Generation : 1500 |  Avg Fitness :  -125  |  Max Fitness :   116  
Generation : 1501 |  Avg Fitness :  -120  |  Max Fitness :   199  
Generation : 1502 |  Avg Fitness :  -122  |  Max Fitness :   132  
Generation : 1503 |  Avg Fitness :  -122  |  Max Fitness :   168  
Generation : 1504 |  Avg Fitness :  -113  |  Max Fitness :   160  
Generation : 1505 |  Avg Fitness :  -112  |  Max Fitness :   135  
Generation : 1506 |  Avg Fitness :  -107  |  Max Fitness :   142  
Generation : 1507 |  Avg Fitness :  -105  |  Max Fitness :   168  
Generation : 1508 |  Avg Fitness :  -118  |  Max Fitness :   192  
Generation : 1509 |  Avg Fitness :  -133  |  Max Fitness :   1

Generation : 1619 |  Avg Fitness :  -122  |  Max Fitness :   251  
Generation : 1620 |  Avg Fitness :  -126  |  Max Fitness :   251  
Generation : 1621 |  Avg Fitness :  -101  |  Max Fitness :   150  
Generation : 1622 |  Avg Fitness :  -110  |  Max Fitness :   183  
Generation : 1623 |  Avg Fitness :  -146  |  Max Fitness :    38  
Generation : 1624 |  Avg Fitness :  -119  |  Max Fitness :   199  
Generation : 1625 |  Avg Fitness :  -119  |  Max Fitness :   109  
Generation : 1626 |  Avg Fitness :  -118  |  Max Fitness :   284  
Generation : 1627 |  Avg Fitness :  -127  |  Max Fitness :   156  
Generation : 1628 |  Avg Fitness :  -108  |  Max Fitness :   267  
Generation : 1629 |  Avg Fitness :  -129  |  Max Fitness :   271  
Generation : 1630 |  Avg Fitness :  -125  |  Max Fitness :   261  
Generation : 1631 |  Avg Fitness :  -105  |  Max Fitness :   267  
Generation : 1632 |  Avg Fitness :  -132  |  Max Fitness :   130  
Generation : 1633 |  Avg Fitness :  -125  |  Max Fitness :   1

Generation : 1742 |  Avg Fitness :  -157  |  Max Fitness :   105  
Generation : 1743 |  Avg Fitness :  -114  |  Max Fitness :   250  
Generation : 1744 |  Avg Fitness :  -126  |  Max Fitness :   254  
Generation : 1745 |  Avg Fitness :   -96  |  Max Fitness :   158  
Generation : 1746 |  Avg Fitness :   -96  |  Max Fitness :   144  
Generation : 1747 |  Avg Fitness :  -137  |  Max Fitness :   127  
Generation : 1748 |  Avg Fitness :  -109  |  Max Fitness :   167  
Generation : 1749 |  Avg Fitness :  -110  |  Max Fitness :   245  
Generation : 1750 |  Avg Fitness :  -118  |  Max Fitness :   252  
Generation : 1751 |  Avg Fitness :  -130  |  Max Fitness :   193  
Generation : 1752 |  Avg Fitness :  -144  |  Max Fitness :   175  
Generation : 1753 |  Avg Fitness :  -133  |  Max Fitness :   218  
Generation : 1754 |  Avg Fitness :  -129  |  Max Fitness :   235  
Generation : 1755 |  Avg Fitness :  -116  |  Max Fitness :   172  
Generation : 1756 |  Avg Fitness :  -133  |  Max Fitness :   2

Generation : 1866 |  Avg Fitness :  -131  |  Max Fitness :   131  
Generation : 1867 |  Avg Fitness :  -143  |  Max Fitness :   119  
Generation : 1868 |  Avg Fitness :  -117  |  Max Fitness :   274  
Generation : 1869 |  Avg Fitness :  -169  |  Max Fitness :   278  
Generation : 1870 |  Avg Fitness :  -136  |  Max Fitness :   251  
Generation : 1871 |  Avg Fitness :  -115  |  Max Fitness :   162  
Generation : 1872 |  Avg Fitness :  -136  |  Max Fitness :   283  
Generation : 1873 |  Avg Fitness :  -161  |  Max Fitness :   142  
Generation : 1874 |  Avg Fitness :  -134  |  Max Fitness :   140  
Generation : 1875 |  Avg Fitness :  -100  |  Max Fitness :   259  
Generation : 1876 |  Avg Fitness :  -119  |  Max Fitness :   279  
Generation : 1877 |  Avg Fitness :  -138  |  Max Fitness :   161  
Generation : 1878 |  Avg Fitness :  -128  |  Max Fitness :   274  
Generation : 1879 |  Avg Fitness :  -139  |  Max Fitness :   282  
Generation : 1880 |  Avg Fitness :  -167  |  Max Fitness :   2

Generation : 1990 |  Avg Fitness :  -125  |  Max Fitness :    34  
Generation : 1991 |  Avg Fitness :  -119  |  Max Fitness :   247  
Generation : 1992 |  Avg Fitness :  -118  |  Max Fitness :   134  
Generation : 1993 |  Avg Fitness :  -131  |  Max Fitness :   139  
Generation : 1994 |  Avg Fitness :   -92  |  Max Fitness :   278  
Generation : 1995 |  Avg Fitness :  -110  |  Max Fitness :   281  
Generation : 1996 |  Avg Fitness :  -113  |  Max Fitness :   268  
Generation : 1997 |  Avg Fitness :  -131  |  Max Fitness :   143  
Generation : 1998 |  Avg Fitness :  -122  |  Max Fitness :   214  
Generation : 1999 |  Avg Fitness :  -138  |  Max Fitness :   231  
Generation : 2000 |  Avg Fitness :  -104  |  Max Fitness :   162  
